# 📦 Inventory & Operations Analytics

This notebook provides comprehensive inventory management and operations analytics for fashion retail. The queries cover inventory health, supply chain efficiency, location performance, demand planning, and operational KPIs.

## 📊 Key Areas Covered:
- **Inventory Health & Optimization**
- **Supply Chain & Movement Analytics** 
- **Location Performance & Operations**
- **Demand Planning & Forecasting**
- **Stockout & Overstock Analysis**
- **Financial Inventory Metrics**

These queries provide actionable insights for:
- Inventory managers optimizing stock levels
- Operations teams managing fulfillment
- Supply chain analysts tracking efficiency
- Financial teams managing inventory investment
- Buyers and planners optimizing assortments

# 1. 📈 Inventory Health & Stock Optimization

In [0]:
-- 1.1 Current Inventory Health Dashboard
WITH current_inventory AS (
    SELECT 
        i.product_key,
        p.sku,
        p.product_name,
        p.category_level_1,
        p.category_level_2,
        p.brand,
        p.price_tier,
        l.location_type,
        l.location_name,
        l.region,
        i.quantity_on_hand,
        i.quantity_available,
        i.quantity_reserved,
        i.quantity_in_transit,
        i.days_of_supply,
        i.stock_cover_days,
        i.inventory_value_cost,
        i.inventory_value_retail,
        i.is_stockout,
        i.is_overstock,
        i.reorder_point,
        i.reorder_quantity
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (
        SELECT MAX(date_key) 
        FROM juan_dev.retail.gold_inventory_fact
    )
    AND p.is_active = true
    AND l.is_active = true
),
inventory_health AS (
    SELECT
        category_level_1,
        category_level_2,
        location_type,
        region,
        COUNT(*) as total_skus,
        COUNT(DISTINCT product_key) as unique_products,
        
        -- Quantity metrics
        SUM(quantity_on_hand) as total_units_on_hand,
        SUM(quantity_available) as total_available_units,
        SUM(quantity_reserved) as total_reserved_units,
        SUM(quantity_in_transit) as total_in_transit_units,
        
        -- Financial metrics
        SUM(inventory_value_cost) as total_cost_value,
        SUM(inventory_value_retail) as total_retail_value,
        SUM(inventory_value_retail) - SUM(inventory_value_cost) as total_markup_value,
        
        -- Health indicators
        SUM(CASE WHEN is_stockout THEN 1 ELSE 0 END) as stockout_count,
        SUM(CASE WHEN is_overstock THEN 1 ELSE 0 END) as overstock_count,
        SUM(CASE WHEN quantity_available <= reorder_point THEN 1 ELSE 0 END) as reorder_needed_count,
        
        -- Performance ratios
        ROUND(SUM(CASE WHEN is_stockout THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as stockout_rate_pct,
        ROUND(SUM(CASE WHEN is_overstock THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as overstock_rate_pct,
        ROUND(SUM(quantity_available) * 100.0 / NULLIF(SUM(quantity_on_hand), 0), 2) as availability_rate_pct,
        
        -- Supply metrics
        AVG(days_of_supply) as avg_days_of_supply,
        AVG(stock_cover_days) as avg_stock_cover_days
    FROM current_inventory
    GROUP BY category_level_1, category_level_2, location_type, region
)
SELECT 
    category_level_1,
    category_level_2,
    location_type,
    region,
    total_skus,
    unique_products,
    total_units_on_hand,
    total_available_units,
    ROUND(total_cost_value, 2) as total_cost_value,
    ROUND(total_retail_value, 2) as total_retail_value,
    ROUND(total_markup_value, 2) as total_markup_value,
    stockout_count,
    overstock_count,
    reorder_needed_count,
    stockout_rate_pct,
    overstock_rate_pct,
    availability_rate_pct,
    ROUND(avg_days_of_supply, 1) as avg_days_supply,
    ROUND(avg_stock_cover_days, 1) as avg_stock_cover
FROM inventory_health
ORDER BY total_retail_value DESC;

In [0]:
-- 1.2 ABC Analysis for Inventory Classification
WITH product_performance AS (
    SELECT 
        p.product_key,
        p.sku,
        p.product_name,
        p.category_level_2,
        p.brand,
        
        -- Sales performance (last 90 days)
        COALESCE(SUM(s.net_sales_amount), 0) as total_revenue_90d,
        COALESCE(SUM(s.quantity_sold), 0) as total_units_sold_90d,
        COALESCE(COUNT(DISTINCT s.date_key), 0) as selling_days,
        
        -- Current inventory investment
        SUM(i.inventory_value_cost) as current_inventory_cost,
        SUM(i.inventory_value_retail) as current_inventory_retail,
        SUM(i.quantity_on_hand) as total_units_on_hand,
        
        -- Calculate turnover and velocity
        CASE 
            WHEN SUM(i.inventory_value_cost) > 0 
            THEN COALESCE(SUM(s.net_sales_amount), 0) * 365.0 / (90 * SUM(i.inventory_value_cost))
            ELSE 0 
        END as inventory_turnover_ratio,
        
        CASE 
            WHEN SUM(i.quantity_on_hand) > 0 
            THEN COALESCE(SUM(s.quantity_sold), 0) * 365.0 / (90 * SUM(i.quantity_on_hand))
            ELSE 0 
        END as inventory_velocity
        
    FROM juan_dev.retail.gold_product_dim p
    LEFT JOIN juan_dev.retail.gold_sales_fact s 
        ON p.product_key = s.product_key 
        AND s.date_key >= (
            SELECT date_key 
            FROM juan_dev.retail.gold_date_dim 
            WHERE calendar_date = DATE_SUB(CURRENT_DATE, 90)
        )
        AND s.is_return = false
    JOIN juan_dev.retail.gold_inventory_fact i 
        ON p.product_key = i.product_key
        AND i.date_key = (
            SELECT MAX(date_key) 
            FROM juan_dev.retail.gold_inventory_fact
        )
    WHERE p.is_active = true
    GROUP BY p.product_key, p.sku, p.product_name, p.category_level_2, p.brand
),
revenue_percentiles AS (
    SELECT 
        *,
        PERCENT_RANK() OVER (ORDER BY total_revenue_90d DESC) as revenue_percentile,
        PERCENT_RANK() OVER (ORDER BY current_inventory_cost DESC) as investment_percentile,
        PERCENT_RANK() OVER (ORDER BY inventory_turnover_ratio DESC) as turnover_percentile
    FROM product_performance
),
abc_classification AS (
    SELECT 
        *,
        CASE 
            WHEN revenue_percentile >= 0.8 THEN 'A - High Revenue'
            WHEN revenue_percentile >= 0.5 THEN 'B - Medium Revenue' 
            ELSE 'C - Low Revenue'
        END as abc_revenue_class,
        
        CASE 
            WHEN turnover_percentile >= 0.8 THEN 'Fast Moving'
            WHEN turnover_percentile >= 0.5 THEN 'Medium Moving'
            ELSE 'Slow Moving'
        END as velocity_class,
        
        CASE 
            WHEN investment_percentile >= 0.8 THEN 'High Investment'
            WHEN investment_percentile >= 0.5 THEN 'Medium Investment'
            ELSE 'Low Investment'
        END as investment_class
    FROM revenue_percentiles
)
SELECT 
    abc_revenue_class,
    velocity_class,
    investment_class,
    COUNT(*) as product_count,
    SUM(total_revenue_90d) as total_revenue,
    SUM(current_inventory_cost) as total_investment,
    SUM(total_units_sold_90d) as total_units_sold,
    SUM(total_units_on_hand) as total_units_on_hand,
    AVG(inventory_turnover_ratio) as avg_turnover_ratio,
    AVG(inventory_velocity) as avg_velocity,
    
    -- Portfolio percentages
    ROUND(SUM(total_revenue_90d) * 100.0 / SUM(SUM(total_revenue_90d)) OVER (), 2) as revenue_share_pct,
    ROUND(SUM(current_inventory_cost) * 100.0 / SUM(SUM(current_inventory_cost)) OVER (), 2) as investment_share_pct
    
FROM abc_classification
GROUP BY abc_revenue_class, velocity_class, investment_class
ORDER BY total_revenue DESC;

# 2. 🚚 Supply Chain & Movement Analytics

An inventory manager would typically ask the following types of questions about this data set, focusing on operational efficiency, stock health, and movement patterns:

Stock Levels & Health

Which products or categories are at risk of stockout or overstock?
What is the current inventory value by location, category, or region?
How many SKUs are below their reorder point and need replenishment?
What is the average days of supply and stock cover for each location or category?
Movement & Supply Chain

What are the most common types and reasons for inventory movements (e.g., receipts, transfers, returns, adjustments)?
What are the top movement flows between locations (e.g., warehouse to store, store to store)?
Are there any bottlenecks or delays in inventory transfers or receipts?
How many units are being moved, and what is the value of these movements by flow or region?
Location Performance

Which locations have the highest or lowest inventory turnover?
How does inventory distribution vary across stores, warehouses, and regions?
Are there locations with consistently high stockouts or overstock rates?
Accuracy & Adjustments

How many inventory adjustments are being made, and what are the main reasons?
What percentage of adjustments are positive (gains) vs. negative (losses)?
Are there patterns in returns or adjustments that indicate process issues?
Operational KPIs

What is the sell-through rate for key products or categories?
How does inventory investment compare to sales performance?
What is the return rate and adjustment rate for each location or category?
Supply Chain Efficiency

How efficient are receipts and transfers (e.g., volume, frequency)?
Are there any regions or flows with high return or adjustment rates?
What is the average movement size and value by type and flow?
These questions help inventory managers optimize stock levels, reduce costs, improve fulfillment, and identify operational issues using the available data set.

In [0]:
-- 2.1 Inventory Movement Flow Analysis
WITH movement_summary AS (
    SELECT 
        im.movement_type,
        im.movement_reason,
        d.fiscal_quarter,
        d.month_name,
        
        -- Location context
        l_from.location_type as from_location_type,
        l_from.region as from_region,
        l_to.location_type as to_location_type,
        l_to.region as to_region,
        
        -- Product context
        p.category_level_1,
        p.category_level_2,
        p.price_tier,
        
        -- Movement metrics
        COUNT(*) as movement_count,
        SUM(im.quantity) as total_units_moved,
        SUM(im.unit_cost * im.quantity) as total_movement_value,
        AVG(im.quantity) as avg_movement_size
        
        -- No timing metrics available
        
    FROM juan_dev.retail.gold_inventory_movement_fact im
    JOIN juan_dev.retail.gold_date_dim d ON im.date_key = d.date_key
    JOIN juan_dev.retail.gold_product_dim p ON im.product_key = p.product_key
    LEFT JOIN juan_dev.retail.gold_location_dim l_from ON im.from_location_key = l_from.location_key
    LEFT JOIN juan_dev.retail.gold_location_dim l_to ON im.to_location_key = l_to.location_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 90)
    GROUP BY 
        im.movement_type, im.movement_reason, d.fiscal_quarter, d.month_name,
        l_from.location_type, l_from.region, l_to.location_type, l_to.region,
        p.category_level_1, p.category_level_2, p.price_tier
)
SELECT 
    movement_type,
    movement_reason,
    CONCAT(COALESCE(from_location_type, 'External'), ' → ', COALESCE(to_location_type, 'External')) as movement_flow,
    CONCAT(COALESCE(from_region, 'External'), ' → ', COALESCE(to_region, 'External')) as region_flow,
    category_level_1,
    price_tier,
    
    SUM(movement_count) as total_movements,
    SUM(total_units_moved) as total_units,
    ROUND(SUM(total_movement_value), 2) as total_value,
    ROUND(AVG(avg_movement_size), 1) as avg_units_per_movement,
    
    -- Movement efficiency
    ROUND(SUM(total_units_moved) / NULLIF(SUM(movement_count), 0), 1) as units_per_transaction,
    ROUND(SUM(total_movement_value) / NULLIF(SUM(total_units_moved), 0), 2) as avg_unit_value
    
FROM movement_summary
GROUP BY 
    movement_type, movement_reason, movement_flow, region_flow,
    category_level_1, price_tier
HAVING SUM(movement_count) > 5
ORDER BY total_value DESC
LIMIT 250;

In [0]:
-- 2.2 Supply Chain Efficiency & Lead Times (fixed: removed unavailable columns)
WITH supply_metrics AS (
    SELECT 
        p.category_level_1,
        p.category_level_2,
        p.brand,
        l.location_type,
        l.region,
        
        -- Receipt analysis (supplier to warehouse)
        COUNT(CASE WHEN im.movement_type = 'receipt' THEN 1 END) as receipts_count,
        SUM(CASE WHEN im.movement_type = 'receipt' THEN im.quantity END) as total_received_units,
        
        -- Transfer analysis (warehouse to store)
        COUNT(CASE WHEN im.movement_type = 'transfer' THEN 1 END) as transfers_count,
        SUM(CASE WHEN im.movement_type = 'transfer' THEN im.quantity END) as total_transferred_units,
            
        -- Returns processing
        COUNT(CASE WHEN im.movement_type = 'return' THEN 1 END) as returns_count,
        SUM(CASE WHEN im.movement_type = 'return' THEN im.quantity END) as total_returned_units,
            
        -- Adjustments (cycle counts, damage, etc.)
        COUNT(CASE WHEN im.movement_type = 'adjustment' THEN 1 END) as adjustments_count,
        SUM(CASE WHEN im.movement_type = 'adjustment' THEN ABS(im.quantity) END) as total_adjusted_units,
        SUM(CASE WHEN im.movement_type = 'adjustment' AND im.quantity > 0 
            THEN im.quantity ELSE 0 END) as positive_adjustments,
        SUM(CASE WHEN im.movement_type = 'adjustment' AND im.quantity < 0 
            THEN ABS(im.quantity) ELSE 0 END) as negative_adjustments
        
    FROM juan_dev.retail.gold_inventory_movement_fact im
    JOIN juan_dev.retail.gold_product_dim p ON im.product_key = p.product_key
    LEFT JOIN juan_dev.retail.gold_location_dim l ON im.to_location_key = l.location_key
    WHERE im.date_key >= (
        SELECT date_key 
        FROM juan_dev.retail.gold_date_dim 
        WHERE calendar_date = date_sub(current_date, 90)
    )
    GROUP BY p.category_level_1, p.category_level_2, p.brand, l.location_type, l.region
)
SELECT 
    category_level_1,
    category_level_2,
    location_type,
    region,
    
    -- Volume metrics
    receipts_count,
    total_received_units,
    transfers_count,
    total_transferred_units,
    returns_count,
    total_returned_units,
    
    -- Accuracy metrics  
    adjustments_count,
    total_adjusted_units,
    ROUND(positive_adjustments * 100.0 / NULLIF(total_adjusted_units, 0), 1) as positive_adj_pct,
    ROUND(negative_adjustments * 100.0 / NULLIF(total_adjusted_units, 0), 1) as negative_adj_pct,
    
    -- Operational ratios
    ROUND(total_returned_units * 100.0 / NULLIF(total_transferred_units, 0), 2) as return_rate_pct,
    ROUND(total_adjusted_units * 100.0 / NULLIF(total_received_units, 0), 2) as adjustment_rate_pct
    
FROM supply_metrics
WHERE receipts_count > 0 OR transfers_count > 0
ORDER BY total_received_units DESC;

# 3. 🏪 Location Performance & Operations

In [0]:
-- 3.1 Location Operational Performance
WITH location_metrics AS (
    SELECT 
        l.location_key,
        l.location_name,
        l.location_type,
        l.region,
        l.selling_sqft,
        l.total_sqft,
        
        -- Current inventory position
        COUNT(DISTINCT i.product_key) as unique_products_stocked,
        SUM(i.quantity_on_hand) as total_units_on_hand,
        SUM(i.quantity_available) as total_available_units,
        SUM(i.inventory_value_cost) as total_inventory_cost,
        SUM(i.inventory_value_retail) as total_inventory_retail,
        
        -- Inventory health
        SUM(CASE WHEN i.is_stockout THEN 1 ELSE 0 END) as stockout_skus,
        SUM(CASE WHEN i.is_overstock THEN 1 ELSE 0 END) as overstock_skus,
        AVG(i.days_of_supply) as avg_days_supply,
        AVG(i.stock_cover_days) as avg_stock_cover,
        
        -- Sales performance (last 30 days)
        COALESCE(sales.transaction_count, 0) as sales_transactions,
        COALESCE(sales.total_revenue, 0) as sales_revenue,
        COALESCE(sales.total_units_sold, 0) as units_sold,
        COALESCE(sales.unique_customers, 0) as unique_customers,
        
        -- Movement activity (last 30 days)
        COALESCE(movements.inbound_movements, 0) as inbound_movements,
        COALESCE(movements.outbound_movements, 0) as outbound_movements,
        COALESCE(movements.inbound_units, 0) as inbound_units,
        COALESCE(movements.outbound_units, 0) as outbound_units
        
    FROM juan_dev.retail.gold_location_dim l
    
    -- Current inventory snapshot
    LEFT JOIN juan_dev.retail.gold_inventory_fact i 
        ON l.location_key = i.location_key
        AND i.date_key = (
            SELECT MAX(date_key) 
            FROM juan_dev.retail.gold_inventory_fact
        )
    
    -- Sales performance (last 30 days)
    LEFT JOIN (
        SELECT 
            location_key,
            COUNT(DISTINCT transaction_id) as transaction_count,
            SUM(net_sales_amount) as total_revenue,
            SUM(quantity_sold) as total_units_sold,
            COUNT(DISTINCT customer_key) as unique_customers
        FROM juan_dev.retail.gold_sales_fact
        WHERE date_key >= (
            SELECT date_key 
            FROM juan_dev.retail.gold_date_dim 
            WHERE calendar_date = DATE_SUB(CURRENT_DATE, 30)
        )
        AND is_return = false
        GROUP BY location_key
    ) sales ON l.location_key = sales.location_key
    
    -- Movement activity (last 30 days)
    LEFT JOIN (
        SELECT 
            COALESCE(to_location_key, from_location_key) as location_key,
            SUM(CASE WHEN to_location_key IS NOT NULL THEN 1 ELSE 0 END) as inbound_movements,
            SUM(CASE WHEN from_location_key IS NOT NULL THEN 1 ELSE 0 END) as outbound_movements,
            SUM(CASE WHEN to_location_key IS NOT NULL THEN quantity ELSE 0 END) as inbound_units,
            SUM(CASE WHEN from_location_key IS NOT NULL THEN quantity ELSE 0 END) as outbound_units
        FROM juan_dev.retail.gold_inventory_movement_fact
        WHERE date_key >= (
            SELECT date_key 
            FROM juan_dev.retail.gold_date_dim 
            WHERE calendar_date = DATE_SUB(CURRENT_DATE, 30)
        )
        GROUP BY COALESCE(to_location_key, from_location_key)
    ) movements ON l.location_key = movements.location_key
    
    WHERE l.is_active = true
    GROUP BY 
        l.location_key, l.location_name, l.location_type, l.region, 
        l.selling_sqft, l.total_sqft, sales.transaction_count, sales.total_revenue,
        sales.total_units_sold, sales.unique_customers, movements.inbound_movements,
        movements.outbound_movements, movements.inbound_units, movements.outbound_units
)
SELECT 
    location_name,
    location_type,
    region,
    
    -- Inventory metrics
    unique_products_stocked,
    total_units_on_hand,
    ROUND(total_inventory_cost, 2) as inventory_investment,
    ROUND(total_inventory_retail, 2) as inventory_retail_value,
    
    -- Health indicators
    stockout_skus,
    overstock_skus,
    ROUND(stockout_skus * 100.0 / NULLIF(unique_products_stocked, 0), 1) as stockout_rate_pct,
    ROUND(overstock_skus * 100.0 / NULLIF(unique_products_stocked, 0), 1) as overstock_rate_pct,
    ROUND(avg_days_supply, 1) as avg_days_supply,
    
    -- Sales performance
    sales_transactions,
    ROUND(sales_revenue, 2) as sales_revenue_30d,
    units_sold as units_sold_30d,
    unique_customers as customers_30d,
    
    -- Movement activity
    inbound_movements + outbound_movements as total_movements_30d,
    inbound_units + outbound_units as total_units_moved_30d,
    
    -- Efficiency ratios
    CASE 
        WHEN selling_sqft > 0 
        THEN ROUND(sales_revenue / selling_sqft, 2) 
        ELSE NULL 
    END as revenue_per_sqft_30d,
    
    CASE 
        WHEN total_inventory_cost > 0 
        THEN ROUND(sales_revenue * 365.0 / (30 * total_inventory_cost), 2)
        ELSE NULL 
    END as inventory_turns_annualized,
    
    ROUND(units_sold * 100.0 / NULLIF(total_available_units, 0), 2) as sell_through_rate_30d_pct
    
FROM location_metrics
ORDER BY 
    CASE location_type 
        WHEN 'store' THEN 1 
        WHEN 'warehouse' THEN 2 
        WHEN 'dc' THEN 3 
        ELSE 4 
    END,
    sales_revenue DESC;

In [0]:
-- 3.2 Cross-Location Inventory Distribution Analysis
WITH product_distribution AS (
    SELECT 
        p.product_key,
        p.sku,
        p.product_name,
        p.category_level_2,
        p.brand,
        p.price_tier,
        
        -- Distribution across location types
        SUM(CASE WHEN l.location_type = 'store' THEN i.quantity_on_hand ELSE 0 END) as store_inventory,
        SUM(CASE WHEN l.location_type = 'warehouse' THEN i.quantity_on_hand ELSE 0 END) as warehouse_inventory,
        SUM(CASE WHEN l.location_type = 'dc' THEN i.quantity_on_hand ELSE 0 END) as dc_inventory,
        SUM(i.quantity_on_hand) as total_network_inventory,
        
        -- Distribution across regions
        SUM(CASE WHEN l.region = 'Northeast' THEN i.quantity_on_hand ELSE 0 END) as northeast_inventory,
        SUM(CASE WHEN l.region = 'Southeast' THEN i.quantity_on_hand ELSE 0 END) as southeast_inventory,
        SUM(CASE WHEN l.region = 'Midwest' THEN i.quantity_on_hand ELSE 0 END) as midwest_inventory,
        SUM(CASE WHEN l.region = 'West' THEN i.quantity_on_hand ELSE 0 END) as west_inventory,
        SUM(CASE WHEN l.region = 'Southwest' THEN i.quantity_on_hand ELSE 0 END) as southwest_inventory,
        
        -- Location counts
        COUNT(DISTINCT CASE WHEN i.quantity_on_hand > 0 THEN l.location_key END) as locations_with_stock,
        COUNT(DISTINCT l.location_key) as total_locations,
        
        -- Stock concentration
        MAX(i.quantity_on_hand) as max_location_stock,
        MIN(CASE WHEN i.quantity_on_hand > 0 THEN i.quantity_on_hand END) as min_location_stock,
        STDDEV(i.quantity_on_hand) as stock_distribution_stddev,
        
        -- Financial distribution
        SUM(i.inventory_value_retail) as total_retail_value
        
    FROM juan_dev.retail.gold_product_dim p
    JOIN juan_dev.retail.gold_inventory_fact i ON p.product_key = i.product_key
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (
        SELECT MAX(date_key) 
        FROM juan_dev.retail.gold_inventory_fact
    )
    AND p.is_active = true
    AND l.is_active = true
    GROUP BY p.product_key, p.sku, p.product_name, p.category_level_2, p.brand, p.price_tier
),
distribution_analysis AS (
    SELECT 
        *,
        -- Location type distribution percentages
        ROUND(store_inventory * 100.0 / NULLIF(total_network_inventory, 0), 1) as store_pct,
        ROUND(warehouse_inventory * 100.0 / NULLIF(total_network_inventory, 0), 1) as warehouse_pct,
        ROUND(dc_inventory * 100.0 / NULLIF(total_network_inventory, 0), 1) as dc_pct,
        
        -- Regional distribution balance (coefficient of variation)
        CASE 
            WHEN total_network_inventory > 0 THEN
                SQRT(
                    POWER(northeast_inventory - total_network_inventory/5.0, 2) +
                    POWER(southeast_inventory - total_network_inventory/5.0, 2) +
                    POWER(midwest_inventory - total_network_inventory/5.0, 2) +
                    POWER(west_inventory - total_network_inventory/5.0, 2) +
                    POWER(southwest_inventory - total_network_inventory/5.0, 2)
                ) / (total_network_inventory/5.0)
            ELSE 0
        END as regional_imbalance_score,
        
        -- Stock coverage
        ROUND(locations_with_stock * 100.0 / NULLIF(total_locations, 0), 1) as location_coverage_pct,
        
        -- Concentration risk (max location as % of total)
        ROUND(max_location_stock * 100.0 / NULLIF(total_network_inventory, 0), 1) as max_concentration_pct
        
    FROM product_distribution
    WHERE total_network_inventory > 0
)
SELECT 
    category_level_2,
    price_tier,
    COUNT(*) as product_count,
    SUM(total_network_inventory) as total_units,
    ROUND(SUM(total_retail_value), 2) as total_value,
    
    -- Average distribution patterns
    ROUND(AVG(store_pct), 1) as avg_store_pct,
    ROUND(AVG(warehouse_pct), 1) as avg_warehouse_pct,
    ROUND(AVG(dc_pct), 1) as avg_dc_pct,
    
    -- Distribution health metrics
    ROUND(AVG(location_coverage_pct), 1) as avg_location_coverage_pct,
    ROUND(AVG(regional_imbalance_score), 2) as avg_regional_imbalance,
    ROUND(AVG(max_concentration_pct), 1) as avg_max_concentration_pct,
    
    -- Risk indicators
    SUM(CASE WHEN location_coverage_pct < 50 THEN 1 ELSE 0 END) as low_coverage_products,
    SUM(CASE WHEN max_concentration_pct > 50 THEN 1 ELSE 0 END) as high_concentration_products,
    SUM(CASE WHEN regional_imbalance_score > 1.5 THEN 1 ELSE 0 END) as imbalanced_products
    
FROM distribution_analysis
GROUP BY category_level_2, price_tier
HAVING COUNT(*) >= 5
ORDER BY total_value DESC;

# 4. 📊 Demand Planning & Forecasting

In [0]:
-- 4.1 Demand Forecast Accuracy Analysis
-- Uses pre-calculated accuracy metrics from gold_demand_forecast_fact
-- Schema: forecast_quantity, actual_quantity, forecast_accuracy, mape, bias, confidence_lower/upper_bound
WITH forecast_data AS (
    SELECT 
        df.forecast_id,
        df.product_key,
        p.sku,
        p.product_name,
        p.category_level_1,
        p.category_level_2,
        p.brand,
        df.location_key,
        l.location_name,
        l.location_type,
        l.region,
        df.forecast_date,
        df.model_type,
        df.model_version,
        
        -- Forecast values
        df.forecast_quantity,
        df.forecast_revenue,
        df.confidence_lower_bound,
        df.confidence_upper_bound,
        df.confidence_level,
        
        -- Actual values (pre-joined in the table)
        df.actual_quantity,
        df.actual_revenue,
        
        -- Pre-calculated accuracy metrics
        df.forecast_accuracy,
        df.mape,
        df.bias,
        
        -- Confidence interval accuracy
        CASE 
            WHEN df.actual_quantity BETWEEN df.confidence_lower_bound AND df.confidence_upper_bound 
            THEN 1 ELSE 0 
        END as within_confidence_interval
        
    FROM juan_dev.retail.gold_demand_forecast_fact df
    JOIN juan_dev.retail.gold_product_dim p ON df.product_key = p.product_key
    JOIN juan_dev.retail.gold_location_dim l ON df.location_key = l.location_key
    WHERE df.forecast_date >= DATE_SUB(CURRENT_DATE, 90)
        AND df.actual_quantity IS NOT NULL  -- Only where actuals are available
),
accuracy_summary AS (
    SELECT 
        category_level_1,
        category_level_2,
        location_type,
        region,
        model_type,
        
        COUNT(*) as forecast_count,
        COUNT(CASE WHEN actual_quantity IS NOT NULL THEN 1 END) as forecasts_with_actuals,
        
        -- Central tendency metrics
        SUM(forecast_quantity) as total_forecasted,
        SUM(actual_quantity) as total_actual,
        AVG(forecast_quantity) as avg_forecasted,
        AVG(actual_quantity) as avg_actual,
        
        -- Accuracy metrics (using pre-calculated values)
        AVG(ABS(forecast_quantity - actual_quantity)) as mean_absolute_error,
        AVG(bias) as mean_bias,
        AVG(mape) as mean_absolute_percentage_error,
        SQRT(AVG(POWER(forecast_quantity - actual_quantity, 2))) as root_mean_squared_error,
        AVG(forecast_accuracy) as avg_forecast_accuracy,
        
        -- Confidence interval performance
        AVG(within_confidence_interval) * 100 as confidence_interval_hit_rate,
        
        -- Directional accuracy
        SUM(CASE 
            WHEN (forecast_quantity > 0 AND actual_quantity > 0) 
              OR (forecast_quantity = 0 AND actual_quantity = 0) 
            THEN 1 ELSE 0 
        END) * 100.0 / COUNT(*) as directional_accuracy_pct
        
    FROM forecast_data
    GROUP BY category_level_1, category_level_2, location_type, region, model_type
)
SELECT 
    category_level_1,
    category_level_2,
    location_type,
    region,
    model_type,
    forecast_count,
    forecasts_with_actuals,
    
    ROUND(total_forecasted, 0) as total_forecasted,
    ROUND(total_actual, 0) as total_actual,
    ROUND((total_forecasted - total_actual) * 100.0 / NULLIF(total_actual, 0), 1) as total_bias_pct,
    
    ROUND(mean_absolute_error, 2) as mae,
    ROUND(mean_bias, 2) as mean_bias,
    ROUND(mean_absolute_percentage_error, 1) as mape_pct,
    ROUND(root_mean_squared_error, 2) as rmse,
    ROUND(avg_forecast_accuracy * 100, 1) as avg_accuracy_pct,
    
    ROUND(confidence_interval_hit_rate, 1) as ci_hit_rate_pct,
    ROUND(directional_accuracy_pct, 1) as directional_accuracy_pct
    
FROM accuracy_summary
WHERE forecasts_with_actuals >= 5
ORDER BY 
    mean_absolute_percentage_error ASC,
    category_level_1, location_type;

In [0]:
-- 4.2 Inventory Planning Recommendations
WITH current_position AS (
    SELECT 
        i.product_key,
        i.location_key,
        p.sku,
        p.product_name,
        p.category_level_2,
        p.brand,
        l.location_name,
        l.location_type,
        l.region,
        
        -- Current inventory status
        i.quantity_on_hand,
        i.quantity_available,
        i.quantity_reserved,
        i.quantity_in_transit,
        i.days_of_supply,
        i.reorder_point,
        i.reorder_quantity,
        i.is_stockout,
        i.is_overstock,
        
        -- Financial metrics
        i.inventory_value_cost,
        i.inventory_value_retail,
        p.base_price,
        p.unit_cost
        
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (
        SELECT MAX(date_key) 
        FROM juan_dev.retail.gold_inventory_fact
    )
    AND p.is_active = true
    AND l.is_active = true
),
sales_velocity AS (
    SELECT 
        s.product_key,
        s.location_key,
        
        -- Recent sales performance (last 30 days)
        SUM(s.quantity_sold) as units_sold_30d,
        COUNT(DISTINCT s.date_key) as selling_days_30d,
        AVG(s.quantity_sold) as avg_daily_sales,
        STDDEV(s.quantity_sold) as sales_volatility,
        
        -- Calculate velocity trends
        SUM(CASE 
            WHEN d.calendar_date >= DATE_SUB(CURRENT_DATE, 15) 
            THEN s.quantity_sold ELSE 0 
        END) * 2 as units_sold_last_15d_annualized,
        
        SUM(CASE 
            WHEN d.calendar_date >= DATE_SUB(CURRENT_DATE, 7) 
            THEN s.quantity_sold ELSE 0 
        END) * 4.28 as units_sold_last_7d_annualized
        
    FROM juan_dev.retail.gold_sales_fact s
    JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
        AND s.is_return = false
    GROUP BY s.product_key, s.location_key
),
future_demand AS (
    -- Uses forecast_quantity from demand forecast table
    -- Aggregates forecasts for next 7 days based on date_key
    SELECT 
        df.product_key,
        df.location_key,
        AVG(df.forecast_quantity) as avg_forecasted_demand_7d,
        SUM(df.forecast_quantity) as total_forecasted_demand_7d
    FROM juan_dev.retail.gold_demand_forecast_fact df
    JOIN juan_dev.retail.gold_date_dim d ON df.date_key = d.date_key
    WHERE d.calendar_date BETWEEN CURRENT_DATE AND DATE_ADD(CURRENT_DATE, 7)
    GROUP BY df.product_key, df.location_key
),
planning_analysis AS (
    SELECT 
        cp.*,
        
        -- Sales metrics
        COALESCE(sv.units_sold_30d, 0) as units_sold_30d,
        COALESCE(sv.avg_daily_sales, 0) as avg_daily_sales,
        COALESCE(sv.sales_volatility, 0) as sales_volatility,
        COALESCE(fd.avg_forecasted_demand_7d, 0) as forecasted_daily_demand,
        
        -- Calculate recommended actions
        CASE 
            WHEN cp.quantity_available <= cp.reorder_point 
                AND COALESCE(sv.avg_daily_sales, 0) > 0 
            THEN cp.reorder_quantity
            WHEN cp.is_stockout 
            THEN GREATEST(cp.reorder_quantity, COALESCE(fd.total_forecasted_demand_7d, 0) * 2)
            ELSE 0
        END as recommended_order_quantity,
        
        -- Risk assessment
        CASE 
            WHEN cp.is_stockout THEN 'CRITICAL - Stockout'
            WHEN cp.quantity_available <= cp.reorder_point THEN 'HIGH - Below Reorder Point'
            WHEN cp.days_of_supply <= 7 THEN 'MEDIUM - Low Stock'
            WHEN cp.is_overstock THEN 'LOW - Overstock'
            ELSE 'NORMAL'
        END as risk_level,
        
        -- Velocity trend
        CASE 
            WHEN COALESCE(sv.units_sold_last_7d_annualized, 0) > COALESCE(sv.units_sold_30d, 0) * 1.2 
            THEN 'ACCELERATING'
            WHEN COALESCE(sv.units_sold_last_7d_annualized, 0) < COALESCE(sv.units_sold_30d, 0) * 0.8 
            THEN 'DECELERATING'
            ELSE 'STABLE'
        END as velocity_trend
        
    FROM current_position cp
    LEFT JOIN sales_velocity sv ON cp.product_key = sv.product_key AND cp.location_key = sv.location_key
    LEFT JOIN future_demand fd ON cp.product_key = fd.product_key AND cp.location_key = fd.location_key
)
SELECT 
    sku,
    product_name,
    location_name,
    location_type,
    region,
    category_level_2,
    
    -- Current status
    quantity_on_hand,
    quantity_available,
    quantity_in_transit,
    days_of_supply,
    risk_level,
    
    -- Sales insights
    units_sold_30d,
    ROUND(avg_daily_sales, 2) as avg_daily_sales,
    ROUND(forecasted_daily_demand, 2) as forecasted_daily_demand,
    velocity_trend,
    
    -- Recommendations
    recommended_order_quantity,
    ROUND(recommended_order_quantity * unit_cost, 2) as recommended_order_value,
    
    -- Financial impact
    ROUND(inventory_value_cost, 2) as current_investment,
    ROUND(
        CASE 
            WHEN avg_daily_sales > 0 
            THEN (base_price - unit_cost) * avg_daily_sales * 30
            ELSE 0 
        END, 2
    ) as potential_monthly_margin
    
FROM planning_analysis
WHERE risk_level IN ('CRITICAL - Stockout', 'HIGH - Below Reorder Point', 'MEDIUM - Low Stock')
   OR recommended_order_quantity > 0
   OR velocity_trend = 'ACCELERATING'
ORDER BY 
    CASE risk_level 
        WHEN 'CRITICAL - Stockout' THEN 1
        WHEN 'HIGH - Below Reorder Point' THEN 2  
        WHEN 'MEDIUM - Low Stock' THEN 3
        ELSE 4
    END,
    potential_monthly_margin DESC
LIMIT 100;

# 5. ⚠️ Stockout & Overstock Analysis

In [0]:
-- 5.1 Stockout Impact and Lost Sales Analysis
WITH stockout_events AS (
    SELECT 
        i.product_key,
        i.location_key,
        p.sku,
        p.product_name,
        p.category_level_2,
        p.brand,
        p.base_price,
        l.location_name,
        l.location_type,
        l.region,
        d.calendar_date as stockout_date,
        d.fiscal_quarter,
        d.is_weekend,
        d.is_holiday,
        i.quantity_on_hand,
        i.quantity_reserved,
        i.is_stockout
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_date_dim d ON i.date_key = d.date_key
    WHERE i.is_stockout = true
        AND d.calendar_date >= DATE_SUB(CURRENT_DATE, 90)
        AND p.is_active = true
),
pre_stockout_sales AS (
    SELECT 
        s.product_key,
        s.location_key,
        AVG(s.quantity_sold) as avg_daily_sales_pre_stockout,
        AVG(s.net_sales_amount) as avg_daily_revenue_pre_stockout,
        STDDEV(s.quantity_sold) as sales_volatility
    FROM juan_dev.retail.gold_sales_fact s
    JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 120)
        AND d.calendar_date < DATE_SUB(CURRENT_DATE, 90)
        AND s.is_return = false
    GROUP BY s.product_key, s.location_key
),
stockout_analysis AS (
    SELECT 
        so.category_level_2,
        so.brand,
        so.location_type,
        so.region,
        so.fiscal_quarter,
        
        -- Stockout frequency metrics
        COUNT(*) as stockout_days,
        COUNT(DISTINCT so.product_key) as products_with_stockouts,
        COUNT(DISTINCT so.location_key) as locations_with_stockouts,
        COUNT(DISTINCT CONCAT(so.product_key, '-', so.location_key)) as unique_product_location_stockouts,
        
        -- Timing analysis
        SUM(CASE WHEN so.is_weekend THEN 1 ELSE 0 END) as weekend_stockouts,
        SUM(CASE WHEN so.is_holiday THEN 1 ELSE 0 END) as holiday_stockouts,
        
        -- Lost sales estimation
        SUM(COALESCE(pss.avg_daily_sales_pre_stockout, 0)) as estimated_lost_units,
        SUM(COALESCE(pss.avg_daily_revenue_pre_stockout, 0)) as estimated_lost_revenue,
        
        -- Financial impact
        AVG(so.base_price) as avg_product_price,
        SUM(COALESCE(pss.avg_daily_sales_pre_stockout, 0) * so.base_price) as estimated_lost_retail_value
        
    FROM stockout_events so
    LEFT JOIN pre_stockout_sales pss 
        ON so.product_key = pss.product_key 
        AND so.location_key = pss.location_key
    GROUP BY 
        so.category_level_2, so.brand, so.location_type, 
        so.region, so.fiscal_quarter
)
SELECT 
    category_level_2,
    location_type,
    region,
    fiscal_quarter,
    
    stockout_days,
    products_with_stockouts,
    locations_with_stockouts,
    unique_product_location_stockouts,
    
    -- Stockout patterns
    ROUND(weekend_stockouts * 100.0 / stockout_days, 1) as weekend_stockout_pct,
    ROUND(holiday_stockouts * 100.0 / stockout_days, 1) as holiday_stockout_pct,
    
    -- Lost sales impact
    ROUND(estimated_lost_units, 0) as est_lost_units,
    ROUND(estimated_lost_revenue, 2) as est_lost_revenue,
    ROUND(estimated_lost_retail_value, 2) as est_lost_retail_value,
    
    -- Average impact per stockout day
    ROUND(estimated_lost_units / NULLIF(stockout_days, 0), 1) as avg_lost_units_per_day,
    ROUND(estimated_lost_revenue / NULLIF(stockout_days, 0), 2) as avg_lost_revenue_per_day
    
FROM stockout_analysis
WHERE stockout_days > 0
ORDER BY estimated_lost_revenue DESC;

In [0]:
-- 5.2 Overstock Analysis and Clearance Opportunities
WITH overstock_analysis AS (
    SELECT 
        i.product_key,
        i.location_key,
        p.sku,
        p.product_name,
        p.category_level_1,
        p.category_level_2,
        p.brand,
        p.season_code,
        p.launch_date,
        p.end_of_life_date,
        p.base_price,
        p.unit_cost,
        l.location_name,
        l.location_type,
        l.region,
        
        -- Current inventory status
        i.quantity_on_hand,
        i.quantity_available,
        i.days_of_supply,
        i.stock_cover_days,
        i.inventory_value_cost,
        i.inventory_value_retail,
        i.is_overstock,
        
        -- Product lifecycle stage
        DATEDIFF(CURRENT_DATE, p.launch_date) as days_since_launch,
        CASE 
            WHEN p.end_of_life_date IS NOT NULL 
            THEN DATEDIFF(p.end_of_life_date, CURRENT_DATE)
            ELSE NULL 
        END as days_until_eol,
        
        -- Recent sales performance (last 30 days)
        COALESCE(recent_sales.units_sold_30d, 0) as units_sold_30d,
        COALESCE(recent_sales.revenue_30d, 0) as revenue_30d,
        COALESCE(recent_sales.avg_daily_sales, 0) as avg_daily_sales,
        COALESCE(recent_sales.selling_days, 0) as selling_days_30d,
        
        -- Sales velocity trends
        COALESCE(recent_sales.units_sold_last_7d, 0) as units_sold_7d,
        COALESCE(recent_sales.units_sold_last_14d, 0) as units_sold_14d
        
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    
    -- Recent sales data
    LEFT JOIN (
        SELECT 
            s.product_key,
            s.location_key,
            SUM(s.quantity_sold) as units_sold_30d,
            SUM(s.net_sales_amount) as revenue_30d,
            AVG(s.quantity_sold) as avg_daily_sales,
            COUNT(DISTINCT s.date_key) as selling_days,
            SUM(CASE 
                WHEN d.calendar_date >= DATE_SUB(CURRENT_DATE, 7) 
                THEN s.quantity_sold ELSE 0 
            END) as units_sold_last_7d,
            SUM(CASE 
                WHEN d.calendar_date >= DATE_SUB(CURRENT_DATE, 14) 
                THEN s.quantity_sold ELSE 0 
            END) as units_sold_last_14d
        FROM juan_dev.retail.gold_sales_fact s
        JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
        WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
            AND s.is_return = false
        GROUP BY s.product_key, s.location_key
    ) recent_sales ON i.product_key = recent_sales.product_key 
        AND i.location_key = recent_sales.location_key
    
    WHERE i.date_key = (
        SELECT MAX(date_key) 
        FROM juan_dev.retail.gold_inventory_fact
    )
    AND p.is_active = true
    AND (
        i.is_overstock = true 
        OR i.days_of_supply > 60
        OR (i.quantity_on_hand > 0 AND recent_sales.units_sold_30d = 0)
    )
),
clearance_recommendations AS (
    SELECT 
        *,
        
        -- Urgency scoring
        CASE 
            WHEN days_until_eol IS NOT NULL AND days_until_eol <= 30 THEN 'URGENT - EOL Soon'
            WHEN units_sold_30d = 0 THEN 'HIGH - No Sales 30d'
            WHEN days_of_supply > 90 THEN 'HIGH - Excess Stock'
            WHEN units_sold_7d = 0 AND units_sold_14d > 0 THEN 'MEDIUM - Slowing'
            WHEN is_overstock THEN 'MEDIUM - Overstock'
            ELSE 'LOW'
        END as clearance_priority,
        
        -- Clearance strategy
        CASE 
            WHEN avg_daily_sales > 0 
            THEN LEAST(quantity_on_hand * 0.5, avg_daily_sales * 14)  -- 2 weeks of sales or 50% of stock
            ELSE quantity_on_hand * 0.3  -- Conservative 30% if no recent sales
        END as recommended_clearance_qty,
        
        -- Financial metrics
        CASE 
            WHEN days_until_eol IS NOT NULL AND days_until_eol <= 30 
            THEN base_price * 0.4  -- 60% off for EOL
            WHEN units_sold_30d = 0 
            THEN base_price * 0.5  -- 50% off for non-movers
            WHEN days_of_supply > 90 
            THEN base_price * 0.6  -- 40% off for excess stock
            ELSE base_price * 0.7  -- 30% off for general overstock
        END as recommended_clearance_price,
        
        -- Potential recovery calculation
        inventory_value_cost - (quantity_on_hand * unit_cost) as carrying_cost_risk
        
    FROM overstock_analysis
)
SELECT 
    sku,
    product_name,
    category_level_2,
    location_name,
    location_type,
    region,
    
    -- Current position
    quantity_on_hand,
    days_of_supply,
    ROUND(inventory_value_cost, 2) as current_investment,
    ROUND(inventory_value_retail, 2) as current_retail_value,
    
    -- Sales performance
    units_sold_30d,
    units_sold_7d,
    ROUND(avg_daily_sales, 2) as avg_daily_sales,
    
    -- Lifecycle
    days_since_launch,
    days_until_eol,
    season_code,
    
    -- Recommendations
    clearance_priority,
    ROUND(recommended_clearance_qty, 0) as recommended_clearance_qty,
    ROUND(base_price, 2) as current_price,
    ROUND(recommended_clearance_price, 2) as recommended_clearance_price,
    ROUND((base_price - recommended_clearance_price) * 100.0 / base_price, 0) as discount_pct,
    
    -- Financial impact
    ROUND(recommended_clearance_qty * recommended_clearance_price, 2) as potential_clearance_revenue,
    ROUND(recommended_clearance_qty * (recommended_clearance_price - unit_cost), 2) as potential_margin,
    ROUND(carrying_cost_risk, 2) as carrying_cost_risk
    
FROM clearance_recommendations
ORDER BY 
    CASE clearance_priority 
        WHEN 'URGENT - EOL Soon' THEN 1
        WHEN 'HIGH - No Sales 30d' THEN 2
        WHEN 'HIGH - Excess Stock' THEN 3
        WHEN 'MEDIUM - Slowing' THEN 4
        WHEN 'MEDIUM - Overstock' THEN 5
        ELSE 6
    END,
    carrying_cost_risk DESC
LIMIT 100;

# 6. 💰 Financial Inventory Metrics

In [0]:
-- 6.1 Inventory Financial Performance Dashboard
WITH inventory_financial_metrics AS (
    SELECT 
        p.category_level_1,
        p.category_level_2,
        p.brand,
        p.price_tier,
        l.location_type,
        l.region,
        
        -- Current inventory investment
        SUM(i.inventory_value_cost) as total_inventory_cost,
        SUM(i.inventory_value_retail) as total_inventory_retail,
        SUM(i.inventory_value_retail - i.inventory_value_cost) as total_inventory_markup,
        SUM(i.quantity_on_hand) as total_units_on_hand,
        
        -- Inventory composition
        COUNT(DISTINCT i.product_key) as unique_products,
        COUNT(*) as total_sku_locations,
        
        -- Health metrics
        SUM(CASE WHEN i.is_stockout THEN i.inventory_value_cost ELSE 0 END) as stockout_investment,
        SUM(CASE WHEN i.is_overstock THEN i.inventory_value_cost ELSE 0 END) as overstock_investment,
        AVG(i.days_of_supply) as avg_days_supply,
        
        -- Sales performance (last 90 days)
        COALESCE(sales_90d.total_revenue, 0) as revenue_90d,
        COALESCE(sales_90d.total_units_sold, 0) as units_sold_90d,
        COALESCE(sales_90d.total_cost_of_goods, 0) as cogs_90d,
        COALESCE(sales_90d.gross_margin, 0) as gross_margin_90d
        
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    
    -- Sales performance for turnover calculations
    LEFT JOIN (
        SELECT 
            s.product_key,
            p_inner.category_level_1,
            p_inner.category_level_2,
            p_inner.brand,
            p_inner.price_tier,
            l_inner.location_type,
            l_inner.region,
            SUM(s.net_sales_amount) as total_revenue,
            SUM(s.quantity_sold) as total_units_sold,
            SUM(s.quantity_sold * p_inner.unit_cost) as total_cost_of_goods,
            SUM(s.net_sales_amount - (s.quantity_sold * p_inner.unit_cost)) as gross_margin
        FROM juan_dev.retail.gold_sales_fact s
        JOIN juan_dev.retail.gold_product_dim p_inner ON s.product_key = p_inner.product_key
        JOIN juan_dev.retail.gold_location_dim l_inner ON s.location_key = l_inner.location_key
        JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
        WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 90)
            AND s.is_return = false
        GROUP BY 
            s.product_key, p_inner.category_level_1, p_inner.category_level_2,
            p_inner.brand, p_inner.price_tier, l_inner.location_type, l_inner.region
    ) sales_90d ON p.category_level_1 = sales_90d.category_level_1
        AND p.category_level_2 = sales_90d.category_level_2
        AND p.brand = sales_90d.brand
        AND p.price_tier = sales_90d.price_tier
        AND l.location_type = sales_90d.location_type
        AND l.region = sales_90d.region
    
    WHERE i.date_key = (
        SELECT MAX(date_key) 
        FROM juan_dev.retail.gold_inventory_fact
    )
    AND p.is_active = true
    AND l.is_active = true
    
    GROUP BY 
        p.category_level_1, p.category_level_2, p.brand, p.price_tier,
        l.location_type, l.region, sales_90d.total_revenue, sales_90d.total_units_sold,
        sales_90d.total_cost_of_goods, sales_90d.gross_margin
),
financial_kpis AS (
    SELECT 
        *,
        
        -- Inventory turns (annualized)
        CASE 
            WHEN total_inventory_cost > 0 
            THEN (cogs_90d * 365.0) / (90 * total_inventory_cost)
            ELSE 0 
        END as inventory_turns_annual,
        
        -- Sell-through rate (90 days annualized)
        CASE 
            WHEN total_units_on_hand > 0 
            THEN (units_sold_90d * 365.0) / (90 * total_units_on_hand) * 100
            ELSE 0 
        END as sell_through_rate_annual_pct,
        
        -- ROI metrics
        CASE 
            WHEN total_inventory_cost > 0 
            THEN (gross_margin_90d * 365.0) / (90 * total_inventory_cost) * 100
            ELSE 0 
        END as inventory_roi_annual_pct,
        
        -- Markup percentages
        CASE 
            WHEN total_inventory_cost > 0 
            THEN total_inventory_markup * 100.0 / total_inventory_cost
            ELSE 0 
        END as markup_percentage,
        
        -- Health ratios
        CASE 
            WHEN total_inventory_cost > 0 
            THEN stockout_investment * 100.0 / total_inventory_cost
            ELSE 0 
        END as stockout_investment_pct,
        
        CASE 
            WHEN total_inventory_cost > 0 
            THEN overstock_investment * 100.0 / total_inventory_cost
            ELSE 0 
        END as overstock_investment_pct
        
    FROM inventory_financial_metrics
)
SELECT 
    category_level_1,
    category_level_2,
    price_tier,
    location_type,
    region,
    
    -- Investment metrics
    ROUND(total_inventory_cost, 2) as inventory_investment,
    ROUND(total_inventory_retail, 2) as inventory_retail_value,
    ROUND(total_inventory_markup, 2) as inventory_markup_value,
    total_units_on_hand,
    unique_products,
    
    -- Performance metrics
    ROUND(revenue_90d, 2) as revenue_90d,
    ROUND(gross_margin_90d, 2) as gross_margin_90d,
    ROUND(inventory_turns_annual, 2) as inventory_turns_annual,
    ROUND(sell_through_rate_annual_pct, 1) as sell_through_rate_pct,
    ROUND(inventory_roi_annual_pct, 1) as inventory_roi_pct,
    
    -- Efficiency metrics
    ROUND(markup_percentage, 1) as markup_pct,
    ROUND(avg_days_supply, 1) as avg_days_supply,
    
    -- Health metrics
    ROUND(stockout_investment, 2) as stockout_investment,
    ROUND(overstock_investment, 2) as overstock_investment,
    ROUND(stockout_investment_pct, 1) as stockout_investment_pct,
    ROUND(overstock_investment_pct, 1) as overstock_investment_pct,
    
    -- Investment concentration
    ROUND(total_inventory_cost / NULLIF(unique_products, 0), 2) as avg_investment_per_product,
    ROUND(revenue_90d / NULLIF(total_inventory_cost, 0), 2) as revenue_to_investment_ratio_90d
    
FROM financial_kpis
WHERE total_inventory_cost > 100  -- Filter out minimal investments
ORDER BY total_inventory_cost DESC;

---

## 📋 Summary

These queries provide comprehensive inventory and operations analytics covering:

**Inventory Health & Optimization:**
- Current inventory status dashboard with health indicators
- ABC analysis for inventory classification and prioritization

**Supply Chain & Movement Analytics:**
- Inventory movement flow analysis across the network
- Supply chain efficiency metrics and lead time analysis

**Location Performance & Operations:**
- Location-specific operational performance metrics
- Cross-location inventory distribution analysis

**Demand Planning & Forecasting:**
- Forecast accuracy analysis with MAPE and bias metrics
- Inventory planning recommendations with risk assessment

**Stockout & Overstock Analysis:**
- Stockout impact and lost sales estimation
- Overstock analysis with clearance recommendations

**Financial Inventory Metrics:**
- Comprehensive financial performance dashboard
- Inventory turns, ROI, and investment efficiency metrics

Each query is optimized for the star schema design and provides actionable insights for inventory managers, operations teams, buyers, and financial analysts to optimize inventory investment and operational efficiency.